# 童话故事自动转短视频

## 1.文字转语言

In [ ]:
!pip install gtts

In [ ]:
from gtts import gTTS
import os

# 要转换为音频的文字
text = "白雪公主在雪地上跳舞"

# 选择语言和速度
language = 'zh'
slow_audio_speed = False

# 使用gTTS创建音频对象
audio_obj = gTTS(text=text, lang=language, slow=slow_audio_speed)

# 保存音频为MP3文件
audio_obj.save("output.mp3")

In [ ]:
from IPython.display import Audio

# 加载并播放MP3文件
Audio("output.mp3")

## 2.文字转图片

In [ ]:
import requests
import json
from PIL import Image
from io import BytesIO
import time

# 创建请求头
headers = {
    "UUID": "afd22831-6e9e-c4e4-2477-825a3328a178"
}

def generate_and_show_image(prompt):
    # 设置请求的URL
    url = "http://midjourney-api.ai-des.com/func2api/Imagine"

    # 创建要发送的数据
    data = {
        "type": "P",
        "prompt": prompt,
        "imgId": "",
        "num": 0
    }

    # 使用requests发送POST请求
    response = requests.post(url, json=data, headers=headers)

    img_id = None
    if response.status_code == 200:
        # 如果响应状态为200(成功)，则解析响应内容
        response_data = response.json()
        img_id = response_data['data']['imgId']
        print(f"Image ID: {img_id}")
    else:
        print(f"请求失败，状态码：{response.status_code}")

    def get_image_url(img_id):
        url = "http://midjourney-api.ai-des.com/func2api/GetPicByImgID"

        data = {
            "imgId": img_id
        }

        while True:
            response = requests.post(url, json=data, headers=headers)
            if response.status_code == 200:
                response_data = response.json()
                img_url = response_data['data']['mjSrcUrl']
                if img_url:  # 图片链接已生成
                    return img_url
            else:
                print(f"请求失败，状态码：{response.status_code}")
            time.sleep(2)  # 每2秒检查一次

    if img_id:
        img_url = get_image_url(img_id)
        print(f"Image URL: {img_url}")

        # 请求图片
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))

        # 展示图片
        img.show()

        # 保存图片
        img.save(f"image_{img_id}.jpg", "JPEG")

# 使用函数生成并显示图片
generate_and_show_image("白雪公主在雪地上跳舞")

## 3.图片转视频

In [ ]:
import cv2
import numpy as np

def create_center_to_full_video(image_path, output_video_path, duration, fps=30):
    img = cv2.imread(image_path)
    height, width, _ = img.shape

    # 初始化视频编写器
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # 计算总帧数
    total_frames = int(duration * fps)

    # 生成逐渐展示的帧
    for frame_no in range(total_frames):
        # 计算缩放因子
        scale_factor = 0.5 + (frame_no / total_frames) * 0.5

        # 缩放图片
        resized_img = cv2.resize(img, (0, 0), fx=scale_factor, fy=scale_factor)

        # 创建黑色背景
        background = np.zeros_like(img)

        # 将缩放后的图片放置在黑色背景的中心
        y_offset = (height - resized_img.shape[0]) // 2
        x_offset = (width - resized_img.shape[1]) // 2
        background[y_offset:y_offset + resized_img.shape[0], x_offset:x_offset + resized_img.shape[1]] = resized_img

        # 将帧写入视频
        video_writer.write(background)

    # 释放视频编写器
    video_writer.release()

image_path = "/content/image_1675686582977400833.jpg"
output_video_path = "output_video.avi"
duration = 3  # 视频持续时间（秒）

create_center_to_full_video(image_path, output_video_path, duration)

In [ ]:
import cv2
import numpy as np

def create_partial_to_full_video(image_path, output_video_path, duration, fps=30):
    img = cv2.imread(image_path)
    height, width, _ = img.shape

    # 初始化视频编写器
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # 计算总帧数
    total_frames = int(duration * fps)

    # 生成逐渐展示的帧
    for frame_no in range(total_frames):
        # 计算裁剪比例
        crop_percentage = 0.5 + (frame_no / total_frames) * 0.5

        # 计算裁剪区域
        y_min = int((1 - crop_percentage) * height // 2)
        y_max = int(height - y_min)
        x_min = int((1 - crop_percentage) * width // 2)
        x_max = int(width - x_min)

        # 裁剪图片
        cropped_img = img[y_min:y_max, x_min:x_max]

        # 缩放图片至原始尺寸
        resized_img = cv2.resize(cropped_img, (width, height))

        # 将帧写入视频
        video_writer.write(resized_img)

    # 释放视频编写器
    video_writer.release()

image_path = "/content/image_1675686582977400833.jpg"
output_video_path = "output_video.avi"
duration = 3  # 视频持续时间（秒）

create_partial_to_full_video(image_path, output_video_path, duration)

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from base64 import b64encode

avi_video_path = "/content/output_video.avi"
mp4_video_path = "/content/output_video.mp4"

# 将AVI视频转换为MP4视频
avi_clip = VideoFileClip(avi_video_path)
avi_clip.write_videofile(mp4_video_path, codec='libx264')

# 读取MP4视频文件并对其进行Base64编码
with open(mp4_video_path, "rb") as video_file:
    video_data = video_file.read()
    video_base64 = b64encode(video_data).decode("ascii")

# 在HTML中嵌入视频播放器
HTML(f"""
<video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
</video>
""")

## 4.音视频合并

In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

video_path = "/content/output_video.avi"
audio_path = "/content/output.mp3"
output_path = "/content/output_combined.mp4"

# 加载视频和音频
video = VideoFileClip(video_path)
audio = AudioFileClip(audio_path)

# 将音频设置为视频的音频
video_with_audio = video.set_audio(audio)

# 保存合并后的音视频文件
video_with_audio.write_videofile(output_path, codec='libx264', audio_codec='aac')

# 关闭视频和音频剪辑
video.close()
audio.close()

Moviepy - Building video /content/output_combined.mp4.
MoviePy - Writing audio in output_combinedTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/output_combined.mp4



Moviepy - Done !
Moviepy - video ready /content/output_combined.mp4


In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_path = "/content/output_combined.mp4"

# 读取视频文件并对其进行Base64编码
with open(video_path, "rb") as video_file:
    video_data = video_file.read()
    video_base64 = b64encode(video_data).decode("ascii")

# 在HTML中嵌入视频播放器
HTML(f"""
<video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
</video>
""")

## 5.故事拆分

许多年以前有一位皇帝，他非常喜欢穿好看的新衣服。他为了要穿得漂亮，把所有的钱都花到衣服上去了，他一点也不关心他的军队，也不喜欢去看戏。除非是为了炫耀一下新衣服，他也不喜欢乘着马车逛公园。他每天每个钟头要换一套新衣服。人们提到皇帝时总是说：“皇上在会议室里。”但是人们一提到他时，总是说：“皇上在室里。”

在他住的那个大城市里，生活很轻松，很愉快。每天有许多外国人到来。有一天来了两个骗子。他们说他们是织工。他们说，他们能织出谁也想象不到的最美丽的布。这种布的色彩和图案不仅是非常好看，而且用它缝出来的衣服还有一种奇异的作用，那就是凡是不称职的人或者愚蠢的人，都看不见这衣服。

“那正是我最喜欢的衣服！”皇帝心里想。“我穿了这样的衣服，就可以看出我的王国里哪些人不称职；我就可以辨别出哪些人是聪明人，哪些人是傻子。是的，我要叫他们马上织出这样的布来！”他付了许多现款给这两个骗子，叫他们马上开始工作。

他们摆出两架织机来，装做是在工作的样子，可是他们的织机上什么东西也没有。他们接二连三地请求皇帝发一些最好的生丝和金子给他们。他们把这些东西都装进自己的腰包，却假装在那两架空空的织机上忙碌地工作，一直忙到深夜。

“我很想知道他们织布究竟织得怎样了，”皇帝想。不过，他立刻就想起了愚蠢的人或不称职的人是看不见这布的。他心里的确感到有些不大自在。他相信他自己是用不着害怕的。虽然如此，他还是觉得先派一个人去看看比较妥当。全城的人都听说过这种布料有一种奇异的力量，所以大家都很想趁这机会来测验一下，看看他们的邻人究竟有多笨，有多傻。

“我要派诚实的老部长到织工那儿去看看，”皇帝想。“只有他能看出这布料是个什么样子，因为他这个人很有头脑，而且谁也不像他那样称职。”

因此这位善良的老部长就到那两个骗子的工作地点去。他们正在空空的织机上忙忙碌碌地工作着。

“这是怎么一回事儿？”老部长想，把眼睛睁得有碗口那么大。

“我什么东西也没有看见！”但是他不敢把这句话说出来。

那两个骗子请求他走近一点，同时问他，布的花纹是不是很美丽，色彩是不是很漂亮。他们指着那两架空空的织机。

这位可怜的老大臣的眼睛越睁越大，可是他还是看不见什么东西，因为的确没有什么东西可看。

“我的老天爷！”他想。“难道我是一个愚蠢的人吗？我从来没有怀疑过我自己。我决不能让人知道这件事。难道我不称职吗？——不成；我决不能让人知道我看不见布料。”

“哎，您一点意见也没有吗？”一个正在织布的织工说。

“啊，美极了！真是美妙极了！”老大臣说。他戴着眼镜仔细地看。“多么美的花纹！多么美的色彩！是的，我将要呈报皇上说我对于这布感到非常满意。”

“嗯，我们听到您的话真高兴，”两个织工一起说。他们把这些稀有的色彩和花纹描述了一番，还加上些名词儿。

这位老大臣注意地听着，以便回到皇帝那里去时，可以照样背得出来。事实上他也就这样办了。

这两个骗子又要了很多的钱，更多的丝和金子，他们说这是为了织布的需要。他们把这些东西全装进腰包里，连一根线也没有放到织机上去。不过他们还是继续在空空的机架上工作。

过了不久，皇帝派了另一位诚实的官员去看看，布是不是很快就可以织好。他的运气并不比头一位大臣的好：他看了又看，但是那两架空空的织机上什么也没有，他什么东西也看不出来。

“您看这段布美不美？”两个骗子问。他们指着一些美丽的花纹，并且作了一些解释。事实上什么花纹也没有。

“我并不愚蠢！”这位官员想，“这大概是因为我不配担当现在这样好的官职吧？这也真够滑稽，但是我决不能让人看出来！”因此他就把他完全没有看见的布称赞了一番，同时对他们说，他非常喜欢这些美丽的颜色和巧妙的花纹。“是的，那真是太美了，”他回去对皇帝说。

城里所有的人都在谈论这美丽的布料。

当这布还在织的时候，皇帝就很想亲自去看一次。他选了一群特别圈定的随员——其中包括已经去看过的那两位诚实的大臣。这样，他就到那两个狡猾的骗子住的地方去。这两个家伙正以全副精神织布，但是一根线的影子也看不见。“您看这不漂亮吗？”那两位诚实的官员说。“陛下请看，多么美丽的花纹！多么美丽的色彩！”他们指着那架空空的织机，因为他们以为别人一定会看得见布料的。

“这是怎么一回事儿呢？”皇帝心里想，“我什么也没有看见！这真是荒唐！难道我是一个愚蠢的人吗？难道我不配做皇帝吗？这真是我从来没有碰见过的一件最可怕的事情。”

“啊，它真是美极了！”皇帝说，“我表示十二分地满意！”

于是他点头表示满意。他装做很仔细地看着织机的样子，因为他不愿意说出他什么也没有看见。跟他来的全体随员也仔细地看了又看，可是他们也没有看出更多的东西。不过，他们也照着皇帝的话说：“啊，真是美极了！”他们建议皇帝用这种新奇的、美丽的布料做成衣服，穿上这衣服亲自去参加快要举行的大典。“真美丽！真精致！真是好极了！”每人都随声附和着。每人都有说不出的快乐。皇帝赐给骗子每人一个爵士的头衔和一枚可以挂在纽扣洞上的勋章；并且还封他们为“御聘织师”。

第二天早晨大典就要举行了。在头天晚上，这两个骗子整夜不睡，点起１６支蜡烛。你可以看到他们是在赶夜工，要完成皇帝的新衣。他们装做把布料从织机上取下来。他们用两把大剪刀在空中裁了一阵子，同时又用没有穿线的针缝了一通。最后，他们齐声说：“请看！新衣服缝好了！”

皇帝带着他的一群最高贵的骑士们亲自到来了。这两个骗子每人举起一只手，好像他们拿着一件什么东西似的。他们说：“请看吧，这是裤子，这是袍子！这是外衣！”等等。“这衣服轻柔得像蜘蛛网一样：穿着它的人会觉得好像身上没有什么东西似的——这也正是这衣服的妙处。”

“一点也不错，”所有的骑士们都说。可是他们什么也没有看见，因为实际上什么东西也没有。

“现在请皇上脱下衣服，”两个骗子说，“我们要在这个大镜子面前为陛下换上新衣。

皇帝把身上的衣服统统都脱光了。这两个骗子装做把他们刚才缝好的新衣服一件一件地交给他。他们在他的腰围那儿弄了一阵子，好像是系上一件什么东西似的：这就是后裾①。皇帝在镜子面前转了转身子，扭了扭腰肢。

“上帝，这衣服多么合身啊！式样裁得多么好看啊！”大家都说。“多么美的花纹！多么美的色彩！这真是一套贵重的衣服！”

“大家已经在外面把华盖准备好了，只等陛下一出去，就可撑起来去！”典礼官说。

“对，我已经穿好了，”皇帝说，“这衣服合我的身么？”于是他又在镜子面前把身子转动了一下，因为他要叫大家看出他在认真地欣赏他美丽的服装。那些将要托着后裾的内臣们，都把手在地上东摸西摸，好像他们真的在拾起后裾似的。他们开步走，手中托着空气——他们不敢让人瞧出他们实在什么东西也没有看见。

这么着，皇帝就在那个富丽的华盖下起来了。站在街上和窗子里的人都说：“乖乖，皇上的新装真是漂亮！他上衣下面的后裾是多么美丽！衣服多么合身！”谁也不愿意让人知道自己看不见什么东西，因为这样就会暴露自己不称职，或是太愚蠢。皇帝所有的衣服从来没有得到这样普遍的称赞。

“可是他什么衣服也没有穿呀！”一个小孩子最后叫出声来。

“上帝哟，你听这个天真的声音！”爸爸说。于是大家把这孩子讲的话私自低声地传播开来。

“他并没有穿什么衣服！有一个小孩子说他并没有穿什么衣服呀！”

“他实在是没有穿什么衣服呀！”最后所有的老百姓都说。皇帝有点儿发抖，因为他似乎觉得老百姓所讲的话是对的。不过他自己心里却这样想：“我必须把这大典举行完毕。”因此他摆出一副更骄傲的神气，他的内臣们跟在他后面走，手中托着一个并不存在的后裾。

In [ ]:
prompt = '''
!生成短视频。 当我发送故事内容并附带这个提示词时，请为我返回视频文稿和多个MidJourney绘画提示词。 示例：


!生成短视频

从前，有一个小村庄，村子里住着一个善良的老木匠和他的孙子。他们过着简单的生活，每天收集木头，制作家具为生。一天，他们在森林里发现了一棵会说话的树……


系统返回内容格式如下：


视频文稿(读给观众看的，要自然，不要说画面、镜头什么的，可以任意人称代入)：

1. 一个宁静的小村庄，阳光洒在青砖绿瓦的房屋上，显得格外美丽。
2. 老木匠和孙子正在木工房里忙碌地制作家具，木屑飞舞，工作氛围和谐愉快。
3. 老木匠和孙子走进森林，绿树成荫，阳光穿过树叶投射在地面上。
4. 他们惊奇地发现一棵会说话的树，树的表情生动，给人一种神奇的感觉。
5. 后续故事情节发展……

MidJourney绘画提示词：

1. 小村庄美景
2. 木工房
3. 老木匠和孙子
4. 森林
5. 会说话的树

'''

In [ ]:
import requests
from IPython.display import display, Markdown
import ipywidgets as widgets

def query_bing_api(message):
    message = prompt + '!生成短视频。 '+ message
    message = message.replace('"', '单引号')
    message = message.replace("'", '双引号')
    message = message.replace("\n", '换行')
    url = "http://api.starchent.top/API/bing.php"
    params = {
        "msg": message,
        "modle": "down",
        "type": "text"
    }

    response = requests.post(url, params=params)
    data = response.text

    if response.status_code == 200:
        return data
    else:
        return "抱歉，无法获取响应内容。"

story = """
许多年以前有一位皇帝，他非常喜欢穿好看的新衣服。他为了要穿得漂亮，把所有的钱都花到衣服上去了，他一点也不关心他的军队，也不喜欢去看戏。除非是为了炫耀一下新衣服，他也不喜欢乘着马车逛公园。他每天每个钟头要换一套新衣服。人们提到皇帝时总是说：“皇上在会议室里。”但是人们一提到他时，总是说：“皇上在室里。”

在他住的那个大城市里，生活很轻松，很愉快。每天有许多外国人到来。有一天来了两个骗子。他们说他们是织工。他们说，他们能织出谁也想象不到的最美丽的布。这种布的色彩和图案不仅是非常好看，而且用它缝出来的衣服还有一种奇异的作用，那就是凡是不称职的人或者愚蠢的人，都看不见这衣服。

“那正是我最喜欢的衣服！”皇帝心里想。“我穿了这样的衣服，就可以看出我的王国里哪些人不称职；我就可以辨别出哪些人是聪明人，哪些人是傻子。是的，我要叫他们马上织出这样的布来！”他付了许多现款给这两个骗子，叫他们马上开始工作。

他们摆出两架织机来，装做是在工作的样子，可是他们的织机上什么东西也没有。他们接二连三地请求皇帝发一些最好的生丝和金子给他们。他们把这些东西都装进自己的腰包，却假装在那两架空空的织机上忙碌地工作，一直忙到深夜。

“我很想知道他们织布究竟织得怎样了，”皇帝想。不过，他立刻就想起了愚蠢的人或不称职的人是看不见这布的。他心里的确感到有些不大自在。他相信他自己是用不着害怕的。虽然如此，他还是觉得先派一个人去看看比较妥当。全城的人都听说过这种布料有一种奇异的力量，所以大家都很想趁这机会来测验一下，看看他们的邻人究竟有多笨，有多傻。

“我要派诚实的老部长到织工那儿去看看，”皇帝想。“只有他能看出这布料是个什么样子，因为他这个人很有头脑，而且谁也不像他那样称职。”

因此这位善良的老部长就到那两个骗子的工作地点去。他们正在空空的织机上忙忙碌碌地工作着。

“这是怎么一回事儿？”老部长想，把眼睛睁得有碗口那么大。

“我什么东西也没有看见！”但是他不敢把这句话说出来。

那两个骗子请求他走近一点，同时问他，布的花纹是不是很美丽，色彩是不是很漂亮。他们指着那两架空空的织机。

这位可怜的老大臣的眼睛越睁越大，可是他还是看不见什么东西，因为的确没有什么东西可看。

“我的老天爷！”他想。“难道我是一个愚蠢的人吗？我从来没有怀疑过我自己。我决不能让人知道这件事。难道我不称职吗？——不成；我决不能让人知道我看不见布料。”

“哎，您一点意见也没有吗？”一个正在织布的织工说。

“啊，美极了！真是美妙极了！”老大臣说。他戴着眼镜仔细地看。“多么美的花纹！多么美的色彩！是的，我将要呈报皇上说我对于这布感到非常满意。”

“嗯，我们听到您的话真高兴，”两个织工一起说。他们把这些稀有的色彩和花纹描述了一番，还加上些名词儿。

这位老大臣注意地听着，以便回到皇帝那里去时，可以照样背得出来。事实上他也就这样办了。

这两个骗子又要了很多的钱，更多的丝和金子，他们说这是为了织布的需要。他们把这些东西全装进腰包里，连一根线也没有放到织机上去。不过他们还是继续在空空的机架上工作。

过了不久，皇帝派了另一位诚实的官员去看看，布是不是很快就可以织好。他的运气并不比头一位大臣的好：他看了又看，但是那两架空空的织机上什么也没有，他什么东西也看不出来。

“您看这段布美不美？”两个骗子问。他们指着一些美丽的花纹，并且作了一些解释。事实上什么花纹也没有。

“我并不愚蠢！”这位官员想，“这大概是因为我不配担当现在这样好的官职吧？这也真够滑稽，但是我决不能让人看出来！”因此他就把他完全没有看见的布称赞了一番，同时对他们说，他非常喜欢这些美丽的颜色和巧妙的花纹。“是的，那真是太美了，”他回去对皇帝说。

城里所有的人都在谈论这美丽的布料。

当这布还在织的时候，皇帝就很想亲自去看一次。他选了一群特别圈定的随员——其中包括已经去看过的那两位诚实的大臣。这样，他就到那两个狡猾的骗子住的地方去。这两个家伙正以全副精神织布，但是一根线的影子也看不见。“您看这不漂亮吗？”那两位诚实的官员说。“陛下请看，多么美丽的花纹！多么美丽的色彩！”他们指着那架空空的织机，因为他们以为别人一定会看得见布料的。

“这是怎么一回事儿呢？”皇帝心里想，“我什么也没有看见！这真是荒唐！难道我是一个愚蠢的人吗？难道我不配做皇帝吗？这真是我从来没有碰见过的一件最可怕的事情。”

“啊，它真是美极了！”皇帝说，“我表示十二分地满意！”

于是他点头表示满意。他装做很仔细地看着织机的样子，因为他不愿意说出他什么也没有看见。跟他来的全体随员也仔细地看了又看，可是他们也没有看出更多的东西。不过，他们也照着皇帝的话说：“啊，真是美极了！”他们建议皇帝用这种新奇的、美丽的布料做成衣服，穿上这衣服亲自去参加快要举行的大典。“真美丽！真精致！真是好极了！”每人都随声附和着。每人都有说不出的快乐。皇帝赐给骗子每人一个爵士的头衔和一枚可以挂在纽扣洞上的勋章；并且还封他们为“御聘织师”。

第二天早晨大典就要举行了。在头天晚上，这两个骗子整夜不睡，点起１６支蜡烛。你可以看到他们是在赶夜工，要完成皇帝的新衣。他们装做把布料从织机上取下来。他们用两把大剪刀在空中裁了一阵子，同时又用没有穿线的针缝了一通。最后，他们齐声说：“请看！新衣服缝好了！”

皇帝带着他的一群最高贵的骑士们亲自到来了。这两个骗子每人举起一只手，好像他们拿着一件什么东西似的。他们说：“请看吧，这是裤子，这是袍子！这是外衣！”等等。“这衣服轻柔得像蜘蛛网一样：穿着它的人会觉得好像身上没有什么东西似的——这也正是这衣服的妙处。”

“一点也不错，”所有的骑士们都说。可是他们什么也没有看见，因为实际上什么东西也没有。

“现在请皇上脱下衣服，”两个骗子说，“我们要在这个大镜子面前为陛下换上新衣。

皇帝把身上的衣服统统都脱光了。这两个骗子装做把他们刚才缝好的新衣服一件一件地交给他。他们在他的腰围那儿弄了一阵子，好像是系上一件什么东西似的：这就是后裾①。皇帝在镜子面前转了转身子，扭了扭腰肢。

“上帝，这衣服多么合身啊！式样裁得多么好看啊！”大家都说。“多么美的花纹！多么美的色彩！这真是一套贵重的衣服！”

“大家已经在外面把华盖准备好了，只等陛下一出去，就可撑起来去！”典礼官说。

“对，我已经穿好了，”皇帝说，“这衣服合我的身么？”于是他又在镜子面前把身子转动了一下，因为他要叫大家看出他在认真地欣赏他美丽的服装。那些将要托着后裾的内臣们，都把手在地上东摸西摸，好像他们真的在拾起后裾似的。他们开步走，手中托着空气——他们不敢让人瞧出他们实在什么东西也没有看见。

这么着，皇帝就在那个富丽的华盖下起来了。站在街上和窗子里的人都说：“乖乖，皇上的新装真是漂亮！他上衣下面的后裾是多么美丽！衣服多么合身！”谁也不愿意让人知道自己看不见什么东西，因为这样就会暴露自己不称职，或是太愚蠢。皇帝所有的衣服从来没有得到这样普遍的称赞。

“可是他什么衣服也没有穿呀！”一个小孩子最后叫出声来。

“上帝哟，你听这个天真的声音！”爸爸说。于是大家把这孩子讲的话私自低声地传播开来。

“他并没有穿什么衣服！有一个小孩子说他并没有穿什么衣服呀！”

“他实在是没有穿什么衣服呀！”最后所有的老百姓都说。皇帝有点儿发抖，因为他似乎觉得老百姓所讲的话是对的。不过他自己心里却这样想：“我必须把这大典举行完毕。”因此他摆出一副更骄傲的神气，他的内臣们跟在他后面走，手中托着一个并不存在的后裾。
"""
res = query_bing_api(story)
print(res)

视频文稿：

1. 许多年以前，有一位皇帝，他的热爱为华美的新衣服，至于王国的事务、军队，甚至举国的节日，都不及那些新衣服来得重要。
2. 这位皇帝生活的大城市里，游客络绎不绝，有一天，两个自称是织工的骗子来了。他们说能织出一种美丽无比，且智者可见，愚者无法看见的布料。
3. 听到这，皇帝非常高兴，他相信有了这样的布料，他就可以看出哪些人是智者，哪些人是愚者。于是，他给这两个骗子提供了大量的金子和丝线，要他们立刻开始织布。
4. 这两个骗子对外宣称他们正在忙碌着织布，却把皇帝提供的金子和丝线全都私吞了。他们在空空的织机上假装完成了这“神奇”的布料。
5. 皇帝派当权的大臣去检查布的织造情况，他们都因不能看见这“神奇”的布，都怀疑自己是愚者，于是都称赞这“神奇”的布的美丽。
6. 皇帝听到大臣们的反馈后，自己也要去看这“神奇”的布料，他也看不见这“神奇”的布料，但他也没有揭穿这个骗局，反而称赞这个“神奇”的布料。
7. 富丽的华盖在街上等着他，准备在大庆典中隆重演出，而皇帝，真的在人民面前裸体出现，都说：“皇帝的新衣多么美丽！”
8. 但是一个小孩子，真实的发出声音：“皇帝并没有穿什么衣服呀！”终于，人们开始醒悟，这是骗局。

MidJourney绘画提示词：

1. 喜欢新衣的皇帝
2. 自称织工的两个骗子
3. 皇帝提供的金子和丝线
4. 空空的织机
5. 派去的大臣
6. 穿上“新衣”的皇帝
7. 赞美之人们
8. 揭穿骗局的小孩子


In [ ]:
def parse_story(content: str):
    # 分割输入内容
    lines = content.split("\n")

    video_script = []
    midjourney_prompts = []

    # 标识当前解析的部分
    current_section = None

    for line in lines:
        if line.strip() == "视频文稿：":
            current_section = "video_script"
            continue
        elif line.strip() == "MidJourney绘画提示词：":
            current_section = "midjourney_prompts"
            continue

        if current_section == "video_script" and line.strip():
            video_script.append(line.strip())
        elif current_section == "midjourney_prompts" and line.strip():
            midjourney_prompts.append(line.strip())

    return [i.split('. ')[1] for i in video_script], [i.split('. ')[1] for i in midjourney_prompts]

In [ ]:
video_script, midjourney_prompts = parse_story(res)

print("视频文稿：")
for item in video_script:
    print(item)

print("\nMidJourney绘画提示词：")
for item in midjourney_prompts:
    print(item)

视频文稿：
许多年以前，有一位皇帝，他的热爱为华美的新衣服，至于王国的事务、军队，甚至举国的节日，都不及那些新衣服来得重要。
这位皇帝生活的大城市里，游客络绎不绝，有一天，两个自称是织工的骗子来了。他们说能织出一种美丽无比，且智者可见，愚者无法看见的布料。
听到这，皇帝非常高兴，他相信有了这样的布料，他就可以看出哪些人是智者，哪些人是愚者。于是，他给这两个骗子提供了大量的金子和丝线，要他们立刻开始织布。
这两个骗子对外宣称他们正在忙碌着织布，却把皇帝提供的金子和丝线全都私吞了。他们在空空的织机上假装完成了这“神奇”的布料。
皇帝派当权的大臣去检查布的织造情况，他们都因不能看见这“神奇”的布，都怀疑自己是愚者，于是都称赞这“神奇”的布的美丽。
皇帝听到大臣们的反馈后，自己也要去看这“神奇”的布料，他也看不见这“神奇”的布料，但他也没有揭穿这个骗局，反而称赞这个“神奇”的布料。
富丽的华盖在街上等着他，准备在大庆典中隆重演出，而皇帝，真的在人民面前裸体出现，都说：“皇帝的新衣多么美丽！”
但是一个小孩子，真实的发出声音：“皇帝并没有穿什么衣服呀！”终于，人们开始醒悟，这是骗局。

MidJourney绘画提示词：
喜欢新衣的皇帝
自称织工的两个骗子
皇帝提供的金子和丝线
空空的织机
派去的大臣
穿上“新衣”的皇帝
赞美之人们
揭穿骗局的小孩子


## 6.视频合并

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

video1_path = "/content/output_video.mp4"
video2_path = "/content/output_combined.mp4"
output_path = "/content/merged_video.mp4"

video1 = VideoFileClip(video1_path)
video2 = VideoFileClip(video2_path)

merged_video = concatenate_videoclips([video1, video2])
merged_video.write_videofile(output_path)

Moviepy - Building video /content/merged_video.mp4.
MoviePy - Writing audio in merged_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/merged_video.mp4



t:  94%|█████████▍| 181/192 [00:28<00:02,  4.12it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/output_combined.mp4, 12582912 bytes wanted but 0 bytes read,at frame 90/102, at time 3.00/3.38 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  95%|█████████▍| 182/192 [00:28<00:02,  4.95it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/output_combined.mp4, 12582912 bytes wanted but 0 bytes read,at frame 91/102, at time 3.03/3.38 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  95%|█████████▌| 183/192 [00:29<00:03,  2.80it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/output_combi

Moviepy - Done !
Moviepy - video ready /content/merged_video.mp4


In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_path = "/content/merged_video.mp4"

# 读取视频文件并对其进行Base64编码
with open(video_path, "rb") as video_file:
    video_data = video_file.read()
    video_base64 = b64encode(video_data).decode("ascii")

# 在HTML中嵌入视频播放器
HTML(f"""
<video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
</video>
""")

## 完整代码(单线程)

In [1]:
!pip install gtts
!pip install pydub

In [ ]:
import requests
from IPython.display import display, Markdown
import ipywidgets as widgets
from gtts import gTTS
import os
import requests
import json
from PIL import Image
from io import BytesIO
import time
from pydub import AudioSegment
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip
from moviepy.editor import VideoFileClip, concatenate_videoclips
from IPython.display import HTML
from base64 import b64encode


prompt = '''
!生成短视频。 当我发送故事内容并附带这个提示词时，请为我返回视频文稿和多个MidJourney绘画提示词。 示例：


!生成短视频

从前，有一个小村庄，村子里住着一个善良的老木匠和他的孙子。他们过着简单的生活，每天收集木头，制作家具为生。一天，他们在森林里发现了一棵会说话的树……


系统返回内容格式如下：


视频文稿(读给观众看的，要自然，不要说画面、镜头什么的，可以任意人称代入)：

1. 一个宁静的小村庄，阳光洒在青砖绿瓦的房屋上，显得格外美丽。
2. 老木匠和孙子正在木工房里忙碌地制作家具，木屑飞舞，工作氛围和谐愉快。
3. 老木匠和孙子走进森林，绿树成荫，阳光穿过树叶投射在地面上。
4. 他们惊奇地发现一棵会说话的树，树的表情生动，给人一种神奇的感觉。
5. 后续故事情节发展……

MidJourney绘画提示词：

1. 小村庄美景
2. 木工房
3. 老木匠和孙子
4. 森林
5. 会说话的树

'''

def query_bing_api(message):
    message = prompt + '!生成短视频。 '+ message
    message = message.replace('"', '单引号')
    message = message.replace("'", '双引号')
    message = message.replace("\n", '换行')
    url = "http://api.starchent.top/API/bing.php"
    params = {
        "msg": message,
        "modle": "down",
        "type": "text"
    }

    response = requests.post(url, params=params)
    data = response.text

    if response.status_code == 200:
        return data
    else:
        return "抱歉，无法获取响应内容。"

def parse_story(content: str):
    # 分割输入内容
    lines = content.split("\n")

    video_script = []
    midjourney_prompts = []

    # 标识当前解析的部分
    current_section = None

    for line in lines:
        if line.strip() == "视频文稿：":
            current_section = "video_script"
            continue
        elif line.strip() == "MidJourney绘画提示词：":
            current_section = "midjourney_prompts"
            continue

        if current_section == "video_script" and line.strip():
            video_script.append(line.strip())
        elif current_section == "midjourney_prompts" and line.strip():
            midjourney_prompts.append(line.strip())

    return [i.split('. ')[1] for i in video_script], [i.split('. ')[1] for i in midjourney_prompts]


# 文字转语言
def text_to_audio(text, language='zh', slow_audio_speed=False):
    audio_obj = gTTS(text=text, lang=language, slow=slow_audio_speed)
    return audio_obj

# 创建请求头
headers = {
    "UUID": "afd22831-6e9e-c4e4-2477-825a3328a178"
}

# 文字转图片
def generate_and_show_image(prompt):
    # 设置请求的URL
    url = "http://midjourney-api.ai-des.com/func2api/Imagine"

    # 创建要发送的数据
    data = {
        "type": "P",
        "prompt": prompt,
        "imgId": "",
        "num": 0
    }

    # 使用requests发送POST请求
    response = requests.post(url, json=data, headers=headers)

    img_id = None
    if response.status_code == 200:
        # 如果响应状态为200(成功)，则解析响应内容
        response_data = response.json()
        img_id = response_data['data']['imgId']
        print(f"Image ID: {img_id}")
    else:
        print(f"请求失败，状态码：{response.status_code}")

    def get_image_url(img_id):
        url = "http://midjourney-api.ai-des.com/func2api/GetPicByImgID"

        data = {
            "imgId": img_id
        }

        while True:
            time.sleep(25)  # 每2秒检查一次
            response = requests.post(url, json=data, headers=headers)
            if response.status_code == 200:
                response_data = response.json()
                img_url = response_data['data']['mjSrcUrl']
                if img_url:  # 图片链接已生成
                    return img_url
            else:
                print(f"请求失败，状态码：{response.status_code}")


    if img_id:
        img_url = get_image_url(img_id)
        print(f"Image URL: {img_url}")

        # 请求图片
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))

        # 展示图片
        img.show()

        return img


def get_audio_length(file_path: str) -> float:
    audio = AudioSegment.from_file(file_path, format="mp3")
    return audio.duration_seconds



def create_partial_to_full_video(image_path, output_video_path, duration, fps=30):
    img = cv2.imread(image_path)
    height, width, _ = img.shape

    # 初始化视频编写器
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # 计算总帧数
    total_frames = int(duration * fps)

    # 生成逐渐展示的帧
    for frame_no in range(total_frames):
        # 计算裁剪比例
        crop_percentage = 0.5 + (frame_no / total_frames) * 0.5

        # 计算裁剪区域
        y_min = int((1 - crop_percentage) * height // 2)
        y_max = int(height - y_min)
        x_min = int((1 - crop_percentage) * width // 2)
        x_max = int(width - x_min)

        # 裁剪图片
        cropped_img = img[y_min:y_max, x_min:x_max]

        # 缩放图片至原始尺寸
        resized_img = cv2.resize(cropped_img, (width, height))

        # 将帧写入视频
        video_writer.write(resized_img)

    # 释放视频编写器
    video_writer.release()

# 代码4
def combine_audio_video(video_path, audio_path, output_path):
    # 加载视频和音频
    video = VideoFileClip(video_path)
    audio = AudioFileClip(audio_path)

    # 将音频设置为视频的音频
    video_with_audio = video.set_audio(audio)

    # 保存合并后的音视频文件
    video_with_audio.write_videofile(output_path, codec='libx264', audio_codec='aac')

    # 关闭视频和音频剪辑
    video.close()
    audio.close()

story = input('请输入故事然后点击回车键')
res = query_bing_api(story)
video_script, midjourney_prompts = parse_story(res)

print("视频文稿：")
for item in video_script:
    print(item)

print("\nMidJourney绘画提示词：")
for item in midjourney_prompts:
    print(item)

videos = []
for i in range(len(video_script)):
    audio_obj = text_to_audio(video_script[i])
    # 保存音频为MP3文件
    audio_obj.save(f"output{i}.mp3")
    audio_length = get_audio_length(f"output{i}.mp3")

    img = generate_and_show_image(midjourney_prompts[i])
    # 保存图片
    img.save(f"image_{i}.jpg", "JPEG")

    create_partial_to_full_video(f"image_{i}.jpg", f"output{i}.avi", round(audio_length,2))

    combine_audio_video(f"output{i}.avi", f"output{i}.mp3", f"output{i}.mp4")
    videos.append(f"output{i}.mp4")


output_path = "merged_video.mp4"

merged_video = concatenate_videoclips([VideoFileClip(video_path) for video_path in videos])
merged_video.write_videofile(output_path)


# 读取视频文件并对其进行Base64编码
with open(output_path, "rb") as video_file:
    video_data = video_file.read()
    video_base64 = b64encode(video_data).decode("ascii")

# 在HTML中嵌入视频播放器
HTML(f"""
<video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
</video>
""")

## 完整代码(多线程)

In [ ]:
import requests
from IPython.display import display, Markdown
import ipywidgets as widgets
from gtts import gTTS
import os
import requests
import json
from PIL import Image
from io import BytesIO
import time
from pydub import AudioSegment
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip
from moviepy.editor import VideoFileClip, concatenate_videoclips
from IPython.display import HTML
from base64 import b64encode
from concurrent.futures import ThreadPoolExecutor


prompt = '''
!生成短视频。 当我发送故事内容并附带这个提示词时，请为我返回视频文稿和多个MidJourney绘画提示词。 示例：


!生成短视频

从前，有一个小村庄，村子里住着一个善良的老木匠和他的孙子。他们过着简单的生活，每天收集木头，制作家具为生。一天，他们在森林里发现了一棵会说话的树……


系统返回内容格式如下：


视频文稿(读给观众看的，要自然，不要说画面、镜头什么的，可以任意人称代入)：

1. 一个宁静的小村庄，阳光洒在青砖绿瓦的房屋上，显得格外美丽。
2. 老木匠和孙子正在木工房里忙碌地制作家具，木屑飞舞，工作氛围和谐愉快。
3. 老木匠和孙子走进森林，绿树成荫，阳光穿过树叶投射在地面上。
4. 他们惊奇地发现一棵会说话的树，树的表情生动，给人一种神奇的感觉。
5. 后续故事情节发展……

MidJourney绘画提示词：

1. 小村庄美景
2. 木工房
3. 老木匠和孙子
4. 森林
5. 会说话的树

'''

def query_bing_api(message):
    message = prompt + '!生成短视频。 '+ message
    message = message.replace('"', '单引号')
    message = message.replace("'", '双引号')
    message = message.replace("\n", '换行')
    url = "http://api.starchent.top/API/bing.php"
    params = {
        "msg": message,
        "modle": "down",
        "type": "text"
    }

    response = requests.post(url, params=params)
    data = response.text

    if response.status_code == 200:
        return data
    else:
        return "抱歉，无法获取响应内容。"

def parse_story(content: str):
    # 分割输入内容
    lines = content.split("\n")

    video_script = []
    midjourney_prompts = []

    # 标识当前解析的部分
    current_section = None

    for line in lines:
        if line.strip() == "视频文稿：":
            current_section = "video_script"
            continue
        elif line.strip() == "MidJourney绘画提示词：":
            current_section = "midjourney_prompts"
            continue

        if current_section == "video_script" and line.strip():
            video_script.append(line.strip())
        elif current_section == "midjourney_prompts" and line.strip():
            midjourney_prompts.append(line.strip())

    return [i.split('. ')[1] for i in video_script], [i.split('. ')[1] for i in midjourney_prompts]


# 文字转语言
def text_to_audio(text, language='zh', slow_audio_speed=False):
    audio_obj = gTTS(text=text, lang=language, slow=slow_audio_speed)
    return audio_obj

# 创建请求头
headers = {
    "UUID": "afd22831-6e9e-c4e4-2477-825a3328a178"
}

# 文字转图片
def generate_and_show_image(prompt):
    # 设置请求的URL
    url = "http://midjourney-api.ai-des.com/func2api/Imagine"

    # 创建要发送的数据
    data = {
        "type": "P",
        "prompt": prompt,
        "imgId": "",
        "num": 0
    }

    # 使用requests发送POST请求
    response = requests.post(url, json=data, headers=headers)

    img_id = None
    if response.status_code == 200:
        # 如果响应状态为200(成功)，则解析响应内容
        response_data = response.json()
        img_id = response_data['data']['imgId']
        print(f"Image ID: {img_id}")
    else:
        print(f"请求失败，状态码：{response.status_code}")

    def get_image_url(img_id):
        url = "http://midjourney-api.ai-des.com/func2api/GetPicByImgID"

        data = {
            "imgId": img_id
        }

        while True:
            time.sleep(25)  # 每2秒检查一次
            response = requests.post(url, json=data, headers=headers)
            if response.status_code == 200:
                response_data = response.json()
                img_url = response_data['data']['mjSrcUrl']
                if img_url:  # 图片链接已生成
                    return img_url
            else:
                print(f"请求失败，状态码：{response.status_code}")


    if img_id:
        img_url = get_image_url(img_id)
        print(f"Image URL: {img_url}")

        # 请求图片
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))

        # 展示图片
        img.show()

        return img


def get_audio_length(file_path: str) -> float:
    audio = AudioSegment.from_file(file_path, format="mp3")
    return audio.duration_seconds



def create_partial_to_full_video(image_path, output_video_path, duration, fps=30):
    img = cv2.imread(image_path)
    height, width, _ = img.shape

    # 初始化视频编写器
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # 计算总帧数
    total_frames = int(duration * fps)

    # 生成逐渐展示的帧
    for frame_no in range(total_frames):
        # 计算裁剪比例
        crop_percentage = 0.5 + (frame_no / total_frames) * 0.5

        # 计算裁剪区域
        y_min = int((1 - crop_percentage) * height // 2)
        y_max = int(height - y_min)
        x_min = int((1 - crop_percentage) * width // 2)
        x_max = int(width - x_min)

        # 裁剪图片
        cropped_img = img[y_min:y_max, x_min:x_max]

        # 缩放图片至原始尺寸
        resized_img = cv2.resize(cropped_img, (width, height))

        # 将帧写入视频
        video_writer.write(resized_img)

    # 释放视频编写器
    video_writer.release()

# 代码4
def combine_audio_video(video_path, audio_path, output_path):
    # 加载视频和音频
    video = VideoFileClip(video_path)
    audio = AudioFileClip(audio_path)

    # 将音频设置为视频的音频
    video_with_audio = video.set_audio(audio)

    # 保存合并后的音视频文件
    video_with_audio.write_videofile(output_path, codec='libx264', audio_codec='aac')

    # 关闭视频和音频剪辑
    video.close()
    audio.close()

story = input('请输入故事然后点击回车键')
res = query_bing_api(story)
video_script, midjourney_prompts = parse_story(res)

print("视频文稿：")
for item in video_script:
    print(item)

print("\nMidJourney绘画提示词：")
for item in midjourney_prompts:
    print(item)

def process_segment(segment):
    print(segment)
    i, (script, prompt) = segment
    audio_obj = text_to_audio(script)
    # 保存音频为MP3文件
    audio_file = f"output{i}.mp3"
    audio_obj.save(audio_file)
    audio_length = get_audio_length(audio_file)

    img = generate_and_show_image(prompt)
    # 保存图片
    img_path = f"image_{i}.jpg"
    img.save(img_path, "JPEG")

    video_path = f"output{i}.avi"
    create_partial_to_full_video(img_path, video_path, round(audio_length, 2))

    final_output_path = f"output{i}.mp4"
    combine_audio_video(video_path, audio_file, final_output_path)

    return final_output_path

segments = list(enumerate(zip(video_script, midjourney_prompts)))
# segments = list(enumerate(zip(video_script, midjourney_prompts)))

with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust number of workers as needed
    video_files = list(executor.map(process_segment, segments))


In [14]:
video_files

['output0.mp4',
 'output1.mp4',
 'output2.mp4',
 'output3.mp4',
 'output4.mp4',
 'output5.mp4']

In [15]:


output_path = "merged_video.mp4"

merged_video = concatenate_videoclips([VideoFileClip(video_path) for video_path in video_files])
merged_video.write_videofile(output_path)


# 读取视频文件并对其进行Base64编码
with open(output_path, "rb") as video_file:
    video_data = video_file.read()
    video_base64 = b64encode(video_data).decode("ascii")

# 在HTML中嵌入视频播放器
HTML(f"""
<video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
</video>
""")

Moviepy - Building video merged_video.mp4.
MoviePy - Writing audio in merged_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video merged_video.mp4



t:  11%|█▏        | 238/2090 [01:17<09:26,  3.27it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file output0.mp4, 12582912 bytes wanted but 0 bytes read,at frame 237/238, at time 7.90/7.92 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  57%|█████▋    | 1193/2090 [06:50<08:53,  1.68it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file output3.mp4, 12582912 bytes wanted but 0 bytes read,at frame 453/454, at time 15.10/15.12 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  84%|████████▍ | 1765/2090 [09:55<01:44,  3.10it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file output4.mp4, 12582912 bytes wanted but 0 bytes

Moviepy - Done !
Moviepy - video ready merged_video.mp4
